In [ ]:
!unzip Text-Detoxification.zip

Archive:  Text-Detoxification.zip
   creating: Text-Detoxification/
   creating: Text-Detoxification/.ipynb_checkpoints/
  inflating: Text-Detoxification/.ipynb_checkpoints/1.0-initial-data-exporation-checkpoint.ipynb  
   creating: Text-Detoxification/data/
   creating: Text-Detoxification/data/raw/
  inflating: Text-Detoxification/data/raw/filtered.tsv  
   creating: Text-Detoxification/models/
   creating: Text-Detoxification/models/runs/
   creating: Text-Detoxification/models/runs/Nov05_08-42-35_DESKTOP-NV6R2CU/
  inflating: Text-Detoxification/models/runs/Nov05_08-42-35_DESKTOP-NV6R2CU/events.out.tfevents.1699162959.DESKTOP-NV6R2CU.596.0  
   creating: Text-Detoxification/models/runs/Nov05_08-44-43_DESKTOP-NV6R2CU/
  inflating: Text-Detoxification/models/runs/Nov05_08-44-43_DESKTOP-NV6R2CU/events.out.tfevents.1699163083.DESKTOP-NV6R2CU.596.1  
   creating: Text-Detoxification/models/runs/Nov05_08-44-46_DESKTOP-NV6R2CU/
  inflating: Text-Detoxification/models/runs/Nov05_08-44-46_D

In [1]:
!pip install transformers
!pip install evaluate
!pip install accelerate -U

In [1]:
import torch
import sys
import pandas as pd
sys.path.append('../')
colab = False
if 'google.colab' in sys.modules:
    # You are running on Google Colab
    colab = True
if colab:
  sys.path.append('Text-Detoxification')

from transformers import BartForConditionalGeneration, BartTokenizer, AutoConfig


In [2]:
from src.data.make_dataset import TextDetoxificationDataset
from src.models.train_model import TrainTextDetoxificationModel

In [3]:
file_path = 'data/raw/filtered.tsv'
if colab:
  file_path = 'Text-Detoxification/' + file_path

model_checkpoint = 'eugenesiow/bart-paraphrase'
df = pd.read_csv(file_path, sep='\t')

config = AutoConfig.from_pretrained(model_checkpoint)
config.max_length = 64

model = BartForConditionalGeneration.from_pretrained(model_checkpoint, config = config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)

In [4]:
from sklearn.model_selection import train_test_split


# Split the dataset into train, test, and validation sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Create datasets using the split data
train_dataset = TextDetoxificationDataset(train_df.reset_index(drop=True), tokenizer, 64)
val_dataset = TextDetoxificationDataset(val_df.reset_index(drop=True), tokenizer, 64)
test_dataset = TextDetoxificationDataset(test_df.reset_index(drop=True), tokenizer, 64)

dataset = {"train": train_dataset, "validation":val_dataset, "test":test_dataset}

In [ ]:
logs = TrainTextDetoxificationModel(model, dataset, tokenizer, 2, 1e-4, 0.01, batch_size = 50)

Epoch,Training Loss,Validation Loss
